Telechargement les bibliothèque utiliser dans la réalisation de projet

la premier est pour converitr le document en image 

la deuxiemme est pour l'extraction de text

In [ ]:
!pip install pdf2image
!pip install pdfminer.six 
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


importation les bibliotheque nécessaire

In [ ]:
import os
from typing import Text
import numpy as np
import re
from PIL import Image,ImageDraw
import cv2 as cv
import json
import time
from pdf2image import convert_from_path


from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io
import sys
import cv2 as cv
import json
import time

import tensorflow as tf

In [ ]:
#root = os.getcwd()[:re.search(r"\bfigure_extraction", os.getcwd()).span()[1]] 
print(os.getcwd())
os.chdir("/content/drive/MyDrive/figure_extraction")

/content


Cette class est pour le but de convertir les documents qui ont des figures en images

cette class il a comme parametre le chemin de document et les images qui contient des figures

In [ ]:
class Convert2image:
    
    def convert(file_source,destination,images):
        path = os.path.join(destination,"Image_with_content")
        os.mkdir(path)
        os.chdir(path)
        root = os.getcwd()
        
        pdf_images = convert_from_path(file_source)
        i=1
            
        for page in pdf_images:
            if i in images:
                page.save('page'+str(i)+'.jpg', 'JPEG')
                
            i=i+1
        
        return True
        

Cette class permet d'extraire le texts de document et l'enregistre dans un fichier text

cette methode contient comme entree le chemin de document
il prend le document et intrepret chaque page pour extraire le text
et à la fin de chaque page en ajouter une chaine de charactere pour indiquer la fin de chaque page

In [ ]:
class Mine_text:
   def get_text(file_source):
        sys.stdin.reconfigure(encoding='utf-8')
        #sys.stdout.reconfigure(encoding='utf-8')
        rsrcmgr = PDFResourceManager()
        retstr = io.StringIO()
        codec = 'utf-8'
        laparams = LAParams(detect_vertical=False)
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        # Create a PDF interpreter object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        # Process each page contained in the document.
        data = list()
        i = 1
        content=""
        
        for page in PDFPage.get_pages(open(file_source, 'rb')):
            
            interpreter.process_page(page)
            
            content += retstr.getvalue().split("\n\n")[len(data)-1].split("\x0c")[1]+"\n\n" if  len(data)!= 0 else ""
            
            for sentence in retstr.getvalue().split("\n\n")[len(data):]:
                content += sentence+"\n\n"
            content +="\n\n***page"+str(i)+" finished***\n\n"
            data = retstr.getvalue().split("\n\n")
            
            i+=1
            
            
        
        return content


In [ ]:
Cette methode utilisé dans le module de conversion 
cette methode permet de trouver les pages qui ont des figures

In [ ]:
#mining the caption of figures from text.txt file and images 
def extract_figtab_caption(type):
    
    source = os.path.join(os.getcwd(),"text_image")
    pdfs = os.listdir(source)
    
    caption_dict=dict()
    special_char = list()
    #special_char_dict = dict()
    if type == "ONE":
        
        
        captions = list()
        caption_dict[pdfs[-1]]=""
        caption = False
        content = ""
        text_file_path = os.path.join(source,pdfs[-1])
        num_page=1
        
        with open(text_file_path+r"/text.txt","r",encoding="utf-8") as file:
            for line in file.readlines():
                for word in line.split():
                    for l in word:
                        if len(str(l.encode())) > 4:
                            if l.encode() not in special_char:
                                special_char.append(l.encode())
                
                if caption :
                    if line[-1] != r"\n" and len(line)>1 :
                        
                        content += line[:-1]+" "
                        
                    else: 
                        
                        caption = False
                        dic = dict()
                        dic["page"] = num_page
                        
                        dic["caption"] = content.strip()
                        content=""
                        
                        captions.append(dic)
                else: 
                    
                    if re.search(r"^(Figure|Fig[.]) ([0-9]+|[IVX]+)(:|[.])",line[:-1]):
                        caption = True
                        content += line[:-1]+" "    
                        
                        
                    else:
                        num_page += 1 if re.match("[*]{3}page[0-9]+ finished[*]{3}",line[:-1]) else 0
                        
        #print(type(page_dict))
        caption_dict[pdfs[-1]] = captions.copy()
        
        
        
        
        
        
        
    else:
        for pdf in pdfs:
            captions = list()
            caption_dict[pdf]=""
            caption = False
            content = ""
            text_file_path = os.path.join(source,pdf)
            num_page=1
            
            with open(text_file_path+r"/text.txt","r",encoding="utf-8") as file:
                for line in file.readlines():
                    for word in line.split():
                        for l in word:
                            if len(str(l.encode())) > 4:
                                if l.encode() not in special_char:
                                    special_char.append(l.encode())
                    
                    if caption :
                        if line[-1] != r"\n" and len(line)>1 :
                            
                            content += line[:-1]+" "
                            
                        else: 
                            
                            caption = False
                            dic = dict()
                            dic["page"] = num_page
                            
                            dic["caption"] = content.strip()
                            content=""
                            
                            captions.append(dic)
                    else: 
                        
                        if re.search(r"^(Figure|Fig[.]) ([0-9]+|[IVX]+)(:|[.])",line[:-1]):
                            caption = True
                            content += line[:-1]+" "    
                            
                            
                        else:
                            num_page += 1 if re.match("[*]{3}page[0-9]+ finished[*]{3}",line[:-1]) else 0
                            
            #print(type(page_dict))
            caption_dict[pdf] = captions.copy()
       
    
    
    return caption_dict

### **Cette methode c'est le module de Conversion**
Cette methode permet utilise les class precedent pour la conversion de document en images
donc il prend le document et extraire le text
apres la methode extract_figtab_caption permet de trouver les pages qui contient des figures et retourne une dictionnaire qui contient la legende de figure avec la page  où se trouve
et a la fin on crée une liste qui contient le numero de page qui contient les figures pour que en convertir que ces images.


In [ ]:


def extract_text_images():
    main_root = os.getcwd()[:re.search(r"\bfigure_extraction", os.getcwd()).span()[1]]
    #main_root = os.getcwd()
    
    pdfs = os.listdir(os.getcwd()+"/pdfs")
    source = os.getcwd()+"/pdfs/"

    for pdf in pdfs:
    
        root = main_root
        destination = os.path.join(os.getcwd(),"text_image/"+pdf.split(".")[0])
        source = os.path.join(os.getcwd(),"pdfs/"+pdf)
       
        
            
        if not os.path.isdir(destination):
            os.mkdir(destination)
            os.chdir(destination)
            
            
            with open("text.txt","w", encoding="utf-8") as file:
                file.write(Mine_text.get_text(source))
                
            os.chdir(root)
            fig_tab = extract_figtab_caption("ONE")
            
            images =  list()
            for pdf_pages in fig_tab:
                for object in fig_tab[pdf_pages]:
                    if object["page"] not in images:
                        images.append(object["page"])
            
            image_size =Convert2image.convert(source,destination,images)
            os.chdir(main_root)
            print("finish with "+pdf)

# **Ce code c'est pour le module de Segmentation**
la methode pre_processing pour la transformation binaire et l'application des filtres morphologiques.
la methode get_contours c'est pour la segmentation.
la methode join_contours c'est pour fusioner les contours qui se croisent
la methode merge_small_element c'esst pour l'affectation des elements petit avec leur proches voisins

In [ ]:



#cette methode est pour transformer l'image en image binaire et l'application des filtres morphlogique
def pre_processing(image):
    
    gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    while not cv.imwrite("gray_result.png",gray):
        pass
    
    thresh = cv.threshold(gray,240,255,cv.THRESH_BINARY_INV)[1]
    while not cv.imwrite("threshold.png",thresh):
        pass
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(14,5))
    dilate = cv.dilate(thresh,kernel,iterations=1)
    while not cv.imwrite("dilate1.png",dilate):
        pass
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(8,5))
    erode = cv.erode(dilate,kernel,iterations=1)
    
    while not cv.imwrite("erode.png",erode):
        pass
    
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(11,6))
    dilate = cv.dilate(erode,kernel,iterations=1)
    
    while not cv.imwrite("dilated2.png",dilate):
        pass
    
    
    return dilate
    
    
    
    
    
    
#create array contours that have x,y coordination and the width and height of the box
#creation une liste des contours proposé et chaque contour a x,y et la hauteur et la largeur
def get_contours(dilate):
    try:
        countours = cv.findContours(dilate,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
        
        countours = countours[0] if len(countours) == 2 else countours[1]
        
        countours = sorted(countours,key = lambda x : cv.boundingRect(x))
        
        real_contours = list()
        for countour in countours:
            x, y, w, h = cv.boundingRect(countour)
            
            real_contours.append([x+3,y+2,w-6,h-7])
            
    except:
        real_contours = []
        countours = []
    return real_contours,countours





#join countours that are intersect or contours that are iside other contours
#cette methode pour regroupé les contours qui se croisent ou superpose
def join_contours(real_contours):
    
    len_countour = len(real_contours)
    breaker = False
    changes = False
    
    for countour in real_contours:
        i = 0
        x, y, w, h = countour 
        
        for j in range(i,len_countour):
            try:
                x2, y2, w2, h2 = real_contours[j]
                if  real_contours[j] is not countour:  
                    breaker = False
                    """
                    x31,y31                           x32,y32




                    x33,y33                          x34,y34
                    """
                    #figure out who's the box that he is in extreme left
                    x31 = min(x,x2)
                    y31 = min(y,y2)
                    if x31 == x : 
                        if y31 == y:
                            #that mean that the box 1 is in extreme left and top
                            #check if the first point off the box 2 is in the area of the box 1
                            if (x2 >= x and x2 <= x+w) and (y2 >= y and y2 <= y+h) :
                                w3 = max(x+w,x2+w2)-x31
                                h3 = max(y+h,y2+h2)-y31
                                
                                real_contours.append([x31,y31,w3,h3])
                                real_contours.remove(real_contours[j])
                                real_contours.remove(countour)
                                breaker = True
                                
                        else :
                            #that mean that the box 1 is in extreme left and box 2 is extreme top
                            if (x2 >= x and x2 <= x+w) and (y2+h2 >= y and y2 <= y) :
                                w3 = max(x+w,x2+w2)-x31
                                h3 = max(y+h,y2+h2)-y31
                                
                                real_contours.append([x31,y31,w3,h3])
                                real_contours.remove(real_contours[j])
                                real_contours.remove(countour)
                                breaker = True
                    else : 
                        if y31 == y2:
                            #that mean that the box 2 is in extreme left and top
                            if (x2 <= x and x <= x2+w2) and (y2 <= y and y <= y2+h2) :
                                w3 = max(x+w,x2+w2)-x31
                                h3 = max(y+h,y2+h2)-y31
                                
                                real_contours.append([x31,y31,w3,h3])
                                real_contours.remove(real_contours[j])
                                real_contours.remove(countour)
                                breaker = True
                        else :
                            #that mean that the box 2 is in extreme left and box 1 is extreme top
                            if (x2 <= x and x <= x2+w2) and (y+h >= y2 and y  <= y2) :
                                w3 = max(x+w,x2+w2)-x31
                                h3 = max(y+h,y2+h2)-y31
                            
                                real_contours.append([x31,y31,w3,h3])
                                real_contours.remove(real_contours[j])
                                real_contours.remove(countour)
                                breaker = True
            except IndexError:
                    pass
                    #print("when we remove two box and add one it cause an error if we are in te last box")
            if breaker : 
                changes = True
                
                break
    
    return changes,real_contours




#this function is for drawnig the boxes in the original image
#cette methode utiliser pour dessiner les countours proposée
def drawing_contours(real_contours,image,source,around_what):
    
    source_searcher = re.search(r"/page[0-9]+[.]jpg",source)
    
    source,image_name = source[:source_searcher.span()[0]],source[source_searcher.span()[0]+1:]
    color = (0,0,255) if around_what == "elements" else (255,0,0) if around_what == "caption" else (0,0,255)
    
    i = 1
    for final_countour in real_contours:
        try:
            x,y,w,h = final_countour
            
            cv.rectangle(image, (x , y) , (x+w , y+h) , color,2)
        except:pass
        
    cv.imwrite(os.path.join(source,image_name.split(".")[0]+"_drawed_"+around_what+"."+image_name.split(".")[1]),image)
        
   
    

#this function for delete drawed box on images
#cette fonction pour supprimer les image qui ont des contours proposée (j'ai utilise cette methode pour faciliter le travaile)
def remove_image_drawed():
    source = os.path.join(os.getcwd(),"text_image")
    images_folder = os.listdir(source)
    for image_folder in images_folder:
        folder_image_with_path = os.path.join(source,image_folder,"Image_with_content")
        
        folder_image_with = os.listdir(folder_image_with_path)
    
        for image in folder_image_with:
            if re.search("drawed",image):
                os.remove(os.path.join(folder_image_with_path,image))
            
       




#Cette methode est crée pour les mots clés de figure qui represente des graphique pour regroupé
#les mot clés et leur figure dans une seule contour
def merge_small_element(contours,merge_what):

    changes = False

    small_contour_to_join = list()
    for contour in contours:
        if merge_what == "final":
            if contour[2]<100 and contour[3]<100 :
                small_contour_to_join.append(contour)
            
        else:
            if (contour[2]<60 and contour[3]<200) or (contour[3]<60 and contour[2]<200) :
                small_contour_to_join.append(contour)
            if contour[2]<5 or contour[3]<5:
                contours.remove(contour)
    
    nearest = list()
    second_nearst = list()
    
    
    for small_contour in small_contour_to_join:
        second_nearst.clear()
        nearest.clear()
        min_dis = 50
        min_seconde = 65
        dis_x = 70
        dis_y = 70
        if merge_what == "final":
            min_dis = int(min_dis/3)
            min_seconde = 0
            dis_x = int(dis_x/3)
            dis_y = int(dis_y/3)
            
            
        if small_contour in contours:
            for contour in contours:
                
                #and contour[2]>60 and contour[3]>60 removed from condition to test 
                if merge_what == "elements":
                    if contour != small_contour and contour[2]>60 and contour[3]>60:
                        
                        if min(contour[0],small_contour[0]) == contour[0]:
                            dis_x = contour[0]+contour[2]-small_contour[0]
                        else : dis_x = small_contour[0]+small_contour[2]-contour[0]
                        
                        
                        
                        dis_y = small_contour[1]-contour[1]
                        
                        dis = abs(dis_x)*1.5 + int(abs(dis_y)*1)
                        #if true contour is right of small_contour
                        if  dis < min_dis :
                            min_dis = dis
                            second_nearst = nearest.copy()
                            nearest = contour.copy()
                        elif dis < min_seconde :
                                min_seconde = dis
                                second_nearst = contour.copy()
                else:
                    if contour != small_contour :
                        
                        if min(contour[0],small_contour[0]) == contour[0]:
                            dis_x = contour[0]+contour[2]-small_contour[0]
                        else : dis_x = small_contour[0]+small_contour[2]-contour[0]
                        
                        
                        
                        dis_y = small_contour[1]-contour[1]
                        
                        dis = abs(dis_x)*1.5 + int(abs(dis_y)*1)
                        #if true contour is right of small_contour
                        if  dis < min_dis :
                            min_dis = dis
                            second_nearst = nearest.copy()
                            nearest = contour.copy()
                        elif dis < min_seconde :
                                min_seconde = dis
                                second_nearst = contour.copy()
                                
        if nearest :
            
            
            if second_nearst: nearest = nearest if nearest[2] < second_nearst[2] else second_nearst
            changes = True
            
            x,y = min(small_contour[0],nearest[0]),min(small_contour[1],nearest[1])
            w,h = max(small_contour[0]+small_contour[2],nearest[0]+nearest[2])-x,max(small_contour[1]+small_contour[3],nearest[1]+nearest[3])-y
            contours.append([x,y,w,h])
            #if nearest in small_contour_to_join:small_contour_to_join.remove(nearest)
            
            if second_nearst and second_nearst is not nearest:
                    x,y = min(small_contour[0],second_nearst[0]),min(small_contour[1],second_nearst[1])
                    w,h = max(small_contour[0]+small_contour[2],second_nearst[0]+second_nearst[2])-x,max(small_contour[1]+small_contour[3],second_nearst[1]+second_nearst[3])-y
                    contours.remove(second_nearst)
                    contours.append([x,y,w,h])
                    
            
            contours.remove(nearest)
            contours.remove(small_contour)
            
            
    if not changes:
        
        small_contour_to_join = list()
        if merge_what=="caption":
            for contour in contours:
                if contour[2] <100 or contour[3]<70:
                    small_contour_to_join.append(contour)
                    
        elif merge_what == "final":
            for contour in contours:
                
                if contour[2]<100 and contour[3]<100 :
                    small_contour_to_join.append(contour)
        else:
            for contour in contours:
                
                if (contour[2]<60 and contour[3]<120) or (contour[3]<60 and contour[2]<120):
                    small_contour_to_join.append(contour)
            
        for small_contour in small_contour_to_join:
            second_nearst.clear()
            nearest.clear()
            
            if merge_what == "caption":
                dis = 50 
                min_seconde = 56
            elif merge_what == "final":
                dis = 30 
                min_seconde = 0
            else:
                min_seconde = 56
                dis = 25 
            if small_contour in contours:
                for contour in contours:
                    #and contour[2]>60 and contour[3]>60
                    if merge_what == "elements":
                        if contour != small_contour and contour[2]>60 and contour[3]>60:
                            
                            if small_contour[0]>=contour[0] and small_contour[0]<=contour[0]+contour[2]:
                                if small_contour[1] < contour[1]:
                                    distance = abs(contour[1]-(small_contour[1]+small_contour[3]))
                                    if distance < dis and distance > 0 :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                                else:
                                    distance =abs(small_contour[1] - (contour[1]+contour[3]))
                                    if  distance < dis and distance > 0  :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                                        
                            if small_contour[1]>=contour[1] and small_contour[1]<=contour[1]+contour[3]:
                                if small_contour[0] < contour[0]:
                                    distance = contour[0]-(small_contour[0]+small_contour[2])
                                    if  distance < dis and distance > 0 :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                                else:
                                    distance = small_contour[0] - (contour[0]+contour[2])
                                    if distance < dis and distance > 0 :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                    else:
                        if contour != small_contour:
                            
                            if small_contour[0]>=contour[0] and small_contour[0]<=contour[0]+contour[2]:
                                if small_contour[1] < contour[1]:
                                    distance = abs(contour[1]-(small_contour[1]+small_contour[3]))
                                    if distance < dis and distance > 0 :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                                else:
                                    distance =abs(small_contour[1] - (contour[1]+contour[3]))
                                    if  distance < dis and distance > 0  :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                                        
                            if small_contour[1]>=contour[1] and small_contour[1]<=contour[1]+contour[3]:
                                if small_contour[0] < contour[0]:
                                    distance = contour[0]-(small_contour[0]+small_contour[2])
                                    if  distance < dis and distance > 0 :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                                else:
                                    distance = small_contour[0] - (contour[0]+contour[2])
                                    if distance < dis and distance > 0 :
                                        dis = distance
                                        second_nearst = nearest.copy()
                                        nearest = contour.copy()
                                    elif distance < min_seconde and distance > 0:
                                        min_seconde = distance
                                        second_nearst = contour.copy()
                    
            if nearest :
                changes = True
                
                if merge_what!="caption": 
                    pass
                if second_nearst: nearest = nearest if nearest[2] < second_nearst[2] else second_nearst
                
                x,y = min(small_contour[0],nearest[0]),min(small_contour[1],nearest[1])
                w,h = max(small_contour[0]+small_contour[2],nearest[0]+nearest[2])-x,max(small_contour[1]+small_contour[3],nearest[1]+nearest[3])-y
                contours.append([x,y,w,h])
                
                if second_nearst and second_nearst is not nearest:
                    x,y = min(small_contour[0],second_nearst[0]),min(small_contour[1],second_nearst[1])
                    w,h = max(small_contour[0]+small_contour[2],second_nearst[0]+second_nearst[2])-x,max(small_contour[1]+small_contour[3],second_nearst[1]+second_nearst[3])-y
                    contours.remove(second_nearst)
                    contours.append([x,y,w,h])
                    
                
                contours.remove(nearest)
                contours.remove(small_contour)
        
    
    return changes,contours


#cette methode pour eliminé les countours qui sont dans autre contours
def join_in (contours):
    
    co = contours.copy()
    for cont in co:
        for c in co:
            
            if c is not cont and c[0]-1<=cont[0] and c[0]+c[2]+1>=cont[0]+cont[2] and c[1]-1<=cont[1] and c[1]+c[3]+1>=cont[1]+cont[3]:
                
                contours.remove(cont)
                
                break
    
    del c
    return contours

"""
[250, 1596, 159, 37]
[488, 1596, 224, 37]
[415, 1589, 72, 74]

"""
#lunch the process
def lunch_extract(source):
    image  = cv.imread(source,cv.IMREAD_COLOR)
    segments = pre_processing(image)
    real_contours,cont = get_contours(segments)
    
    
    while join_contours(real_contours)[0]:
        pass
    """
    """
    #eliminé les elemennts  qui sont trés petit
    con = real_contours.copy()
    for contour in con:
        if contour[3] <7 or contour[2] <7:
            real_contours.remove(contour)
    del con
    
    
    while merge_small_element(real_contours,"elements")[0]:
        pass
    
    """
    while join_contours(real_contours)[0]:
        pass
    """
    

    real_contours = join_in(real_contours)
    
    con = real_contours.copy()
    for contour in con:
        if contour[3] <40 or contour[2] <40:
            real_contours.remove(contour)
    del con
    
    drawing_contours(real_contours,image,source,"elements")
    return real_contours

"""
if __name__ == "__main__":
    #remove_image_drawed() 
 
    #lunch_extract("C:\\Users\\LENOVO THINKPAD X260\\pretraitement_pdf\\text_image\\aaai10_1\\Image_with_content\\page7.jpg")

"""

'\nif __name__ == "__main__":\n    #remove_image_drawed() \n \n    #lunch_extract("C:\\Users\\LENOVO THINKPAD X260\\pretraitement_pdf\\text_image\\aaai10_1\\Image_with_content\\page7.jpg")\n\n'

Cette fonction pour automatiser le module de segmentation sur tous les images des documents

In [ ]:
def extract_image_element(figure_pages):
   
    remove_image_drawed()
    source = os.path.join(os.getcwd(),"text_image")
    
    contour = dict()
    
   
    
    for pdf_name in figure_pages:
        pages = dict()
    
        image_with_content_path = os.path.join(source,pdf_name,"Image_with_content")
        images = os.listdir(image_with_content_path)
        image_with_content = dict()
        for image in images:
            image_with_content[re.findall("[0-9]+",image.split(".")[0])[0]] = image
        for page in figure_pages[pdf_name]:
            
            pages[image_with_content[str(page)].split(".")[0]] = lunch_extract(os.path.join(image_with_content_path,image_with_content[str(page)]))

        contour[pdf_name]=dict(pages)
    del pages
    
    with open("contour.json","w") as contour_file:
        json.dump(contour,contour_file,indent=2)
        
    
    return contour

Preparation de dataset de test de tout l'application
pour le dataset cs-150



In [ ]:


with open(os.getcwd()+"/conference/annotations.json","r",encoding="utf-8") as anno_file:
    
    anno_dict = json.load(anno_file)
    new_anno = dict()
    for anno in anno_dict:
        
        del anno_dict[anno]["pages_annotated"]
        #figures = anno_dict[anno]
        new_anno[anno] = list()
        for figure in anno_dict[anno]["figures"]:
            
            if figure["figure_type"] != "Figure" :
                
                del figure
            else:
                del figure["name"]
                del figure["caption"]
                del figure["caption_bb"]
                del figure["dpi"]
                del figure["figure_type"]
                
                box = figure["region_bb"]
                
                box[2] = int((box[2]-box[0])*612/figure["page_width"])
                box[3] = int((box[3]-box[1])*792/figure["page_height"])
                box[0] = int(box[0]*612/figure["page_width"])
                box[1] = int(box[1]*792/figure["page_height"])
                
                figure["page_height"] = 792
                figure["page_width"] = 612
                new_anno[anno].append(figure)


with open(os.getcwd()+"/conference/Test.json","w") as test_file:
    json.dump(new_anno,test_file,indent=2)

# **notre application**
combine les 2 module précédent avec le module de classification

In [ ]:
class Extractor:


    #cette fonction pour preparé les regions proposé afin de passer au model
    def load(image):
        #image = Image.open(filename)
        image = image.convert("RGB")
        np_image = image.resize((128,128))
        np_image = np.expand_dims(np_image, axis=0)
        return np_image
    

    #cette fonction pour dessiner les regions qui sont predire comme figure
    def drawing(dic):
        
        for doc in dic:
            print(doc)
            pages = os.listdir("text_image/"+doc+"/Image_with_content")
            color = (0,255,0)
            for page in pages:
                if page.split(".")[0] in dic[doc].keys():
                    image = Image.open("text_image/"+doc+"/Image_with_content/"+page)
                    image = image.resize((612,792))
                    img = ImageDraw.Draw(image)  
                    
                    for con in dic[doc][page.split(".")[0]]:
                        
                        
                        img.rectangle([(con[0], con[1]), (con[0]+con[2] , con[1]+con[3] )], outline ="red")
                    image.save("text_image/"+doc+"/Image_with_content/drawed"+page)
            pass


if __name__ == "__main__":

    start = time.time()
    #le module de conversion
    extract_text_images()
    
    #trouver les page qui ont des figures
    caption_dict = extract_figtab_caption("ALL")
    
    
    figures_pages = dict()
    
    for pdf_name in caption_dict.keys():
        name = pdf_name
        figures_pages[pdf_name] = []
        if caption_dict[pdf_name] != []:
            for figures in caption_dict[pdf_name]:
                
                if figures["page"] not in figures_pages[pdf_name]:
                    figures_pages[pdf_name].append(figures["page"])
        else:pass
    #segmentation des images
    dictionary = extract_image_element(figures_pages)
    del figures_pages
    end = time.time()
    print(end-start)
    
    
    
    
    
    
    #le module de classification
    img_height = 128
    img_width = 128
    
    model = tf.keras.Sequential([
    
    tf.keras.Input(shape=(img_height, img_width, 3)),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),

    # Block 1
    #best tmp
    #tf.keras.layers.Conv2D(64, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.MaxPooling2D((2,2)),

    # Block 2
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.MaxPooling2D((2,2)),

    # Block 3
    tf.keras.layers.Conv2D(256, (5, 5), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.MaxPooling2D((2,2)),

    # Flatten
    tf.keras.layers.Flatten(),

    # Fully connected layers
    tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
   

    checkpoint_filepath ='tmp/checkpoint'
    try:
        model.load_weights(checkpoint_filepath)
        print("***************\n\nload worked\n\n**************")
    except:
        print("***************\n\nno load worked\n\n**************")
        pass
    
    
    
    
    
    fig_dict = dict()
    
    #prediction des regions génére par le module de segmentation par le module de classification
    for pages in dictionary.items():
        name = pages[0]
        file_path = os.path.join(os.getcwd(),"text_image",pages[0],"Image_with_content")
        fig_dict_fig = dict()
        fig_dict[name] = fig_dict_fig
        
        for page in pages[1].items():
            
            fig_dict_fig[page[0]] = list()
            image_page = Image.open(os.path.join(file_path,str(page[0])+".jpg"))
            image_page = image_page.resize((612,792))
            
            for box in page[1]:
                
                
                box[0] = int((box[0])*612/1700)
                box[1] = int((box[1])*792/2200)
                box[2] = int((box[2])*612/1700)
                box[3] = int((box[3])*792/2200)
                
                sub_image = image_page.crop((box[0],box[1],box[0]+box[2],box[1]+box[3]))
                
                image = Extractor.load(sub_image)
                if 1-model.predict(image)[0][0]>= 0.5:
                    print(1-model.predict(image)[0][0])
                    
                    fig_dict_fig[page[0]].append(box)
        del fig_dict_fig
    
    #print(fig_dict)
   
    
  
    
    #regroupé les figures qui se croissent et qui sont petit
    #si il ya encore des elements petit donc c'est elements sont isolée donc la plupart sont des equations
    for doc in fig_dict:
        for page in fig_dict[doc]:
            
            while join_contours(fig_dict[doc][page])[0]:
                pass 
            
            while merge_small_element(fig_dict[doc][page],"final")[0]:
                pass
            
            contours = fig_dict[doc][page].copy()
            for cont in contours:
                if cont[2] <40 or cont[3] <40:
                    fig_dict[doc][page].remove(cont)
                    
    #dessiner les contours sur les figures extrait par notre solution
    Extractor.drawing(fig_dict)
    
    

finish with pdf2.0.pdf
9.564416408538818
***************

load worked

**************
0.9999936254785098
0.9999333486630348
0.5588083565235138
0.9993778169155121
0.9999283696088241
0.9995037615299225
0.9997531771659851
0.9999754074888187
0.9990599453449249
0.9998283982276917
0.9947647452354431
0.9613203406333923
0.9999684227113903
0.9997839033603668
0.9988441467285156
0.9662812948226929
0.9993137121200562
0.9992939829826355
0.9897597134113312
0.9997596740722656
0.9988691806793213
0.9999996426663529
0.9998738467693329
0.999969042622979
0.9999999669780486
aaai10_2
pdf2
